<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/OpenNMT_GRU_ManythingsDataset_Per_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install OpenNMT**

In [2]:
!pip install --upgrade pip
!pip install OpenNMT-tf

     |████████████████████████████████| 1.5MB 3.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 145 kB 4.2 MB/s 
     |████████████████████████████████| 75.5 MB 25 kB/s 
     |████████████████████████████████| 2.5 MB 82.1 MB/s 
     |████████████████████████████████| 269 kB 75.9 MB/s 
     |████████████████████████████████| 65 kB 3.6 MB/s 
     |████████████████████████████████| 703 kB 71.6 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=193adba2837e84dbbbb304c377c68876c046a16b6e736de47f157dcb6b178b33
  Stored in directory: /root/.cache/pip/wheels/e5/9d/ad/2ee53cf262cba1ffd8afe1487eef788ea3f260b7e6232a80fc
Successfully built pyyaml
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.

### **OpenNMT version**

In [3]:
!onmt-main -v

2021-01-23 06:47:46.117154: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
OpenNMT-tf 2.14.0


### **Preparing Dataset**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!wget "https://www.manythings.org/anki/pes-eng.zip"

--2021-01-23 06:56:49--  https://www.manythings.org/anki/pes-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112553 (110K) [application/zip]
Saving to: ‘pes-eng.zip’

pes-eng.zip         100%[===================>] 109.92K  --.-KB/s    in 0.1s    

2021-01-23 06:56:50 (884 KB/s) - ‘pes-eng.zip’ saved [112553/112553]



In [10]:
!unzip pes-eng.zip -d "/content/"

Archive:  pes-eng.zip
  inflating: /content/_about.txt     
  inflating: /content/pes.txt        


In [12]:
path = "/content/pes.txt"

In [43]:
f = open(path, "r")
print(f.read(978)) 

Who?	چه کسی؟	CC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #4746184 (mahdiye)
Go on.	ادامه بده ( ادامه دادن )	CC-BY 2.0 (France) Attribution: tatoeba.org #2230774 (CK) & #6669169 (arashorosia)
Smile.	لبخند بزن.	CC-BY 2.0 (France) Attribution: tatoeba.org #2764108 (CK) & #4746196 (mahdiye)
Attack!	حمله!	CC-BY 2.0 (France) Attribution: tatoeba.org #1972610 (CK) & #4746181 (mahdiye)
Got it!	گرفتم!	CC-BY 2.0 (France) Attribution: tatoeba.org #320484 (CM) & #888645 (mahdiye)
I know.	من می دانم.	CC-BY 2.0 (France) Attribution: tatoeba.org #319990 (CK) & #888643 (mahdiye)
Listen.	گوش کن.	CC-BY 2.0 (France) Attribution: tatoeba.org #1913088 (CK) & #4746178 (mahdiye)
Really?	واقعا؟	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #5118169 (Hussein64)
Really?	جدا؟	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #5118171 (Hussein64)
Why me?	چرا من؟	CC-BY 2.0 (France) Attribution: tatoeba.org #24958 (CK) & #4746261 (mahdiye)



In [40]:
import re

In [53]:
text_file = open("/content/English.txt", "w")
text_file2 = open("/content/Persian.txt", "w")

f = open(path, "r")
for r in f:
  text_file.write(re.split(r'\t+', r)[0]+"\n") # English txt
  text_file2.write(re.split(r'\t+', r)[1]+"\n") # Persian txt

In [54]:
text_file.close()
text_file2.close()